In [1]:
import pandas as pd
import numpy as np
import requests
 


# https://open.toronto.ca/dataset/ttc-bus-delay-data/

#### Get data from toronto TTC open data resource API

In [2]:
# toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# to hit our API, the URL is:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# datasets are called "packages". Each package can contain many "resources"
# to retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = { "id": "ttc-bus-delay-data"}
package = requests.get(url, params = params).json()

# to get resource data:

resource_list = []

for idx, resource in enumerate(package["result"]["resources"]):

    # to get metadata for non datastore_active resources:
    if not resource["datastore_active"]:
        url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
        resource_metadata = requests.get(url).json()

        resource_list.append(resource_metadata)
        
        # use the "url" attribute to download this file


#### Get the previous FULL year's dataset e.g. if this year is 2023, get 2022's data since it has the full year.

In [3]:
# read in data and parse datetime from separate date and time columns

ttc_delay = pd.read_excel(resource_list[-2]['result']['url'], parse_dates=[['Date','Time']])

C:\Users\42069v3\AppData\Local\Temp\ipykernel_19180\3273399316.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ttc_delay = pd.read_excel(resource_list[-2]['result']['url'], parse_dates=[['Date','Time']])


In [4]:
ttc_delay

,Date_Time,Route,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle
0,2022-01-01 02:00:00,320,Saturday,YONGE AND DUNDAS,General Delay,0,0,NaN,8531
1,2022-01-01 02:00:00,325,Saturday,OVERLEA AND THORCLIFFE,Diversion,131,161,W,8658
2,2022-01-01 02:00:00,320,Saturday,YONGE AND STEELES,Operations - Operator,17,20,S,0
3,2022-01-01 02:07:00,320,Saturday,YONGE AND STEELES,Operations - Operator,4,11,S,0
4,2022-01-01 02:13:00,320,Saturday,YONGE AND STEELES,Operations - Operator,4,8,S,0
...,...,...,...,...,...,...,...,...,...
58702,2022-12-31 23:21:00,32,Saturday,EGLINTON AND EDITH (W,Mechanical,2,24,W,1085
58703,2022-12-31 23:28:00,113,Saturday,DANFORTH AND DAWES,Vision,20,40,W,8589
58704,2022-12-31 23:56:00,72,Saturday,LOWER SHERBOURNE AND L,Security,18,36,N,8493
58705,2022-12-31 00:17:00,22,Saturday,BINGHAM LOOP,Cleaning - Unsanitary,10,20,S,8582


In [5]:
ttc_delay = ttc_delay.sort_values(by='Date_Time').reset_index()

If Date_Time, Location, Incident, Route, Vehicle, Direction are duplicated, can infer that the min delay, min gap, are misinputs or multiple inputs. If location, direction, vehicle, route are the same, it's likely that the rows are referring to the same incident.

In [6]:
ttc_delay = ttc_delay.drop_duplicates(subset=['Date_Time','Location', 'Incident','Route','Vehicle','Direction'])

sum(ttc_delay.duplicated(subset=['Date_Time','Location', 'Incident','Route','Vehicle','Direction']))

0

In [7]:
# get basic description of dataset

ttc_delay.describe()

,index,Date_Time,Min Delay,Min Gap,Vehicle
count,58298.000000,58298,58298.000000,58298.000000,58298.000000
mean,29346.770335,2022-07-04 00:07:38.932381952,20.163419,32.722718,5469.564445
min,0.000000,2022-01-01 00:28:00,0.000000,0.000000,0.000000
25%,14647.250000,2022-04-06 18:04:00,9.000000,17.000000,1553.000000
50%,29333.500000,2022-07-12 21:00:30,11.000000,22.000000,7959.000000
75%,44038.750000,2022-09-27 14:29:45,20.000000,36.000000,8546.000000
max,58706.000000,2022-12-31 23:56:00,999.000000,999.000000,93561.000000
std,16957.353745,NaN,49.109166,50.796621,4361.400924


In [8]:
# check shape of dataset

ttc_delay.shape

(58298, 10)

In [9]:
# get idea of how many nulls there are in dataset

ttc_delay.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58298 entries, 0 to 58706
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   index      58298 non-null  int64         
 1   Date_Time  58298 non-null  datetime64[ns]
 2   Route      57915 non-null  object        
 3   Day        58298 non-null  object        
 4   Location   58298 non-null  object        
 5   Incident   58298 non-null  object        
 6   Min Delay  58298 non-null  int64         
 7   Min Gap    58298 non-null  int64         
 8   Direction  47885 non-null  object        
 9   Vehicle    58298 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 4.9+ MB


In [10]:
# count unique values in incident type 

ttc_delay.Incident.value_counts()

Incident
Operations - Operator               19306
Mechanical                          16384
Collision - TTC                      3512
Security                             3374
Utilized Off Route                   3225
General Delay                        3205
Diversion                            2881
Emergency Services                   2420
Cleaning - Unsanitary                1548
Investigation                         906
Vision                                705
Road Blocked - NON-TTC Collision      624
Held By                               162
Late Entering Service                  40
Cleaning - Disinfection                 6
Name: count, dtype: int64

#### Notice that there's duplicate values that aren't counted together e.g. DUNDAS AND YONGE and YONGE AND DUNDAS. Make a method to remove this duplication.

In [11]:
# count unique values in delay location

ttc_delay.Location.value_counts()

Location
KENNEDY STATION           1371
KIPLING STATION           1268
PIONEER VILLAGE STATIO    1130
FINCH STATION             1090
EGLINTON STATION          1045
                          ... 
75 COSBURN AVE               1
SHERBOURNE AND MAPLE A       1
SHEPPARD AND WILSON HI       1
VICTORIA PARK AND FER        1
EGLINTON AND EDITH (W        1
Name: count, Length: 10028, dtype: int64

In [12]:
def comb_add(add):
  # combines a list of addresses into single string with AND in between
  if len(add) == 2:
    return add[0] + ' AND ' + add[1]
  return add[0]

ttc_delay.Location = ttc_delay.Location.str.split(' AND ')

ttc_delay.Location = ttc_delay.Location.apply(sorted).apply(comb_add)
ttc_delay.Location.value_counts()

Location
KENNEDY STATION           1371
KIPLING STATION           1268
PIONEER VILLAGE STATIO    1130
FINCH STATION             1090
EGLINTON STATION          1045
                          ... 
518 ELLERSLIE AVE            1
OVERSKATE AND WILSON         1
PIONEER VILLAGE STATI        1
HUMBER TO OSSINGTON ST       1
EDITH (W AND EGLINTON        1
Name: count, Length: 9195, dtype: int64

#### Add only date and only time for tableau fields

In [13]:
# preparing for tableau 

ttc_delay['Date'] = pd.to_datetime(ttc_delay['Date_Time']).dt.date
ttc_delay['Time'] = pd.to_datetime(ttc_delay['Date_Time']).dt.time

In [14]:
# ttc_delay.to_excel('ttc_delay_2022.xlsx', sheet_name='Data')

In [15]:
# correlation plot and EDA

import sklearn.corr_